In [1]:
import sys
sys.path.append(r"D:\Akash\Work\AI\model_evaluation_VSCODE\yolov5")  # Adjust this path based on your folder structure


In [22]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [23]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom',path=r"D:\Akash\Work\AI\XAI\weights\content\yolov5\runs\train\exp\weights\best.pt", force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to D:\Softwares\LaMa\lama\hub\master.zip
YOLOv5  2024-8-5 Python-3.8.10 torch-2.4.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [24]:
%pwd

'd:\\Akash\\Work\\AI\\model_evaluation_VSCODE'

In [17]:
import torch
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import non_max_suppression
from torchcam.methods import SmoothGradCAMpp
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import WindowsPath

# Load the YOLOv5 model with custom weights
model_path = "D:\\Akash\\Work\\AI\\XAI\\weights\\content\\yolov5\\runs\\train\\exp\\weights\\best.pt"
model = DetectMultiBackend(model_path)
model.eval()  # Set model to evaluation mode


Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs


DetectMultiBackend(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv3): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): Sequential(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(

In [21]:
print(last_conv_layer)


Detect(
  (m): ModuleList(
    (0): Conv2d(128, 18, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(256, 18, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2d(512, 18, kernel_size=(1, 1), stride=(1, 1))
  )
)


In [19]:
# Create the Grad-CAM object using torchcam
last_conv_layer = model.model.model[-1] if hasattr(model.model, 'model') else model.model[-1]
cam_extractor = SmoothGradCAMpp(last_conv_layer)  # Last convolutional layer

def apply_gradcam(image, model, cam_extractor, threshold=0.5):
    """
    Applies Grad-CAM on YOLOv5 detections.
    """
    # Preprocess the image
    image_tensor = torch.from_numpy(image).float().permute(2, 0, 1) / 255.0
    image_tensor = image_tensor.unsqueeze(0)

    # Get model predictions
    with torch.no_grad():
        outputs = model(image_tensor)

    # Apply non-max suppression to filter boxes
    detections = non_max_suppression(outputs, threshold)

    # Initialize Grad-CAM mask
    mask = np.zeros_like(image[:, :, 0])

    # Loop through detections and apply Grad-CAM
    for det in detections[0]:
        if det is not None and len(det) > 0:
            for *box, conf, cls in det:
                # Generate Grad-CAM heatmap
                cam_map = cam_extractor(outputs[0].squeeze(0)[int(cls)])
                mask += cam_map.numpy()

    # Overlay the heatmap on the original image
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    overlayed_image = cv2.addWeighted(image, 0.5, heatmap, 0.5, 0)

    return overlayed_image



RuntimeError: Given groups=1, weight of size [18, 128, 1, 1], expected input[1, 3, 224, 224] to have 128 channels, but got 3 channels instead